# 2.7_GCD問題への挑戦

In [ ]:
3
1 3 5
-2 4 1

In [ ]:
5
1 2 3 4 5 
1 0 1 0 1

In [1]:
n = 3
x = [1, 3, 5]
y = [-2, 4, 1]

In [29]:
n = 5
x = [1, 2, 3, 4, 5]
y = [1, 0, 1, 0, 1]

In [15]:
def min_inner_product_smart(n, x, y):
    x.sort()
    y.sort(reverse=True)
    return sum([x[i]*y[i] for i in range(n)])
    

In [30]:
min_inner_product_smart(n, x, y)

6

In [23]:
import itertools
import numpy as np
import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

In [40]:
def min_inner_product_brute_force(n, x, y):
    best = np.inf
    best_index = None
    for index in itertools.permutations(range(n), n):
        tmp = sum([x[i] * y[index[i]] for i in range(n)])
        if tmp < best:
            best = tmp
            best_index = index
        tmp = min(tmp, best)
    return best, [y[i] for i in best_index] 

In [41]:
min_inner_product_brute_force(n, x, y)

(6, [1, 1, 1, 0, 0])

# Problem 2

In [103]:
N = 3
arr = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0],
])

In [43]:
arr

array([[0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]])

## 方針
- 各行を入れ替えるのみ
- 各行について、0でない index の最大値をスコアとする
- スコアの小さい順に並べる(入れ替え)
- 並べ替えの回数をカウントする

In [73]:
def score(row):
    ''' 0 でない index の最大値
    '''
    row = (row != 0).tolist()
    # index を取るために、ひっくり返して、最初のを取る
    row.reverse()
    index = row.index(1)
    return len(row)-index-1
    

In [102]:
def shuffle_wide(scores):
    ''' 隣接しない行も入れ替える
    '''
    # 動かす
    N = len(scores)
    shuffle = 0
    for i in range(N):
        i_index = scores.index(min(scores[i:]))
        if i_index == i:
            continue
        if i_index > i:
            RuntimeError("Can't make lower triangle.")
        # shuffle
        scores[i], scores[i_index] = scores[i_index], scores[i]
        shuffle += 1
    return shuffle, scores

In [104]:
# shuffle_neighbor
N = len(scores)
scores = []
for row in arr:
    scores.append(score(row))


In [111]:
def shuffle_neighbor(scores):
    N = len(scores)
    shuffle = 0
    for i in range(N):
        i_index = scores.index(min(scores[i:]))

        # 1行ずつ入れ替える
        for j in range(i_index, i, -1):
            scores[j], scores[j-1] = scores[j-1], scores[j]
            shuffle += 1
    return shuffle, scores

In [117]:
N = 3
arr = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0],
])
calc_shuffle(arr)

(2, [0, 1, 2])

In [97]:
scores

[0, 1, 2]

In [119]:
# スコア計算
def calc_scores(arr):
    scores = []
    for row in arr:
        scores.append(score(row))
    return scores


In [124]:
N = 4
arr = np.array([
    [1,1,1,0,],
    [1,1,0,0,],
    [1,1,0,0,],
    [1,0,0,0,],
])
scores = calc_scores(arr)
shuffle_neighbor(scores)

(4, [0, 1, 2, 1])

In [123]:
scores

[2, 1, 1, 0]

# 囚人の問題
- 証明はできないが、中間で割るのが正しい気がする

- 割る板の数を決める len(N) - a_n - 1, a_n - a_n-1 -1, ..., a_0 - 0 - 1,
- 隣合う板の中で、足すと最も小さいものを取り出し、足して、元に代入する. 足したものは保存.
- 長さが1になったら終わり
- heapq の問題に近い

In [67]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

from typing import List 

In [61]:
P = 20
Q = 3
A = [3, 6, 14]

In [62]:
# 計
total = 0

# 端を入れる
A = [0] + A + [P+1]

# 割る長さを決める
lengths = [A[i+1]-A[i]-1 for i in range(Q+1)]

In [63]:
lengths

[2, 2, 7, 6]

In [64]:
while(len(lengths) > 1):
    # 隣あう板の中で、最も小さいものを取り出し、足して、元に入れる
    sum_values = [lengths[i]+lengths[i+1] for i in range(len(lengths)-1)]
    min_index = sum_values.index(min(sum_values))
    tmp = lengths.pop(min_index)
    
    # 切る長さには自分を含めない
    total += lengths[min_index] + tmp
    
    # 元の長さには自分を含める
    lengths[min_index] = tmp + lengths[min_index] + 1
    logger.debug(f"Lengths: {lengths}")

lengths
total

DEBUG:root:Lengths: [5, 7, 6]
DEBUG:root:Lengths: [13, 6]
DEBUG:root:Lengths: [20]


35

In [68]:
def prisoners(P, Q, A: List):
    # 計
    total = 0

    # 端を入れる
    A = [0] + A + [P+1]

    # 割る長さを決める
    lengths = [A[i+1]-A[i]-1 for i in range(Q+1)]
    
    # 長さ1になるまで続ける
    while(len(lengths) > 1):
        # 隣あう板の中で、最も小さいものを取り出し、足して、元に入れる
        sum_values = [lengths[i]+lengths[i+1] for i in range(len(lengths)-1)]
        min_index = sum_values.index(min(sum_values))
        tmp = lengths.pop(min_index)

        # 切る長さには自分を含めない
        total += lengths[min_index] + tmp

        # 元の長さには自分を含める
        lengths[min_index] = tmp + lengths[min_index] + 1
        logger.debug(f"Lengths: {lengths}")
    
    return total
    

In [69]:
prisoners(8, 1, [3])

DEBUG:root:Lengths: [8]


7

In [70]:
prisoners(20, 3, [3, 6, 14])

DEBUG:root:Lengths: [5, 7, 6]
DEBUG:root:Lengths: [13, 6]
DEBUG:root:Lengths: [20]


35

In [35]:
tmp = lengths.pop(min_index)

In [36]:
lengths[min_index] = tmp + lengths[min_index]

In [37]:
total += lengths[min_index]

In [38]:
total

4

In [39]:
lengths

[4, 7, 6]